In [1]:
import numpy as np
import scipy as sp

In [33]:
def int_cuadrada_inf(f,a,b,N=100):
    Delta = (b-a)/N
    x_i = np.linspace(a,b,N+1)
    # f tiene que ser una función vectorizada
    f_xi = f(x_i)
    # estimación inferior
    area = np.sum(f_xi[:-1])*Delta
    return area
def int_cuadrada_sup(f,a,b,N=100):
    Delta = (b-a)/N
    x_i = np.linspace(a,b,N+1)
    # f tiene que ser una función vectorizada
    f_xi = f(x_i)
    # estimación superior
    area = np.sum(f_xi[1:])*Delta
    return area
def int_cuadrada_media(f,a,b,N=100):
    Delta = (b-a)/N
    x_i = np.linspace(a,b,N+1)
    # f tiene que ser una función vectorizada
    f_xi = f(x_i+Delta/2)
    # estimación media excluye el último punto
    area = np.sum(f_xi[:-1])*Delta
    return area
def int_cuadrada_trapecio(f,a,b,N=100):
    Delta = (b-a)/N
    x_i = np.linspace(a,b,N+1)
    # f tiene que ser una función vectorizada
    f_xi = f(x_i)
    # estimación media excluye el último punto
    area = (0.5*f_xi[0]+np.sum(f_xi[1:-1])+0.5*f_xi[-1])*Delta
    return area
def int_cuadrada_trapecio2(f,a,b,N=100):
    area = (int_cuadrada_inf(f,a,b,N)+int_cuadrada_sup(f,a,b,N))/2
    return area

In [36]:
%%timeit
# nuestra función a integrar es sqrt(tan(x))
int_cuadrada_trapecio(np.sqrt,0,1,N=1000000) ### R// 2/3

15.8 ms ± 487 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [37]:
%%timeit
# nuestra función a integrar es sqrt(tan(x))
int_cuadrada_trapecio2(np.sqrt,0,1,N=1000000) ### R// 2/3

33.9 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
np.linspace(0,1,11)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [16]:
def int_simpson(f,a,b,N=100):
    assert N%2==0
    assert a<b
    assert callable(f)
    u2_3 = 2/3
    u4_3 = 4/3
    x_i = np.linspace(a,b,N+1)
    delta = (b-a)/N
    f_xi = f(x_i)
    f_xi[0] /=2; f_xi[-1] /=2
    area = delta*np.sum([
        (u2_3*f_ if index[0]%2==0 else u4_3*f_)
        for index,f_ in np.ndenumerate(f_xi)
    ])
    return area

In [9]:
def sqrtTan(x):
    return np.sqrt(np.tan(x))
V_sqrtTan = np.vectorize(sqrtTan)

In [24]:
epsilon=1/10000;int_simpson(sqrtTan,0,np.pi/2-epsilon,N=1000)

2.2203835974064865

In [ ]:
epsilon = 1-np.linspace(0,1,100001)
intsqrttan = np.array([int_simpson(sqrtTan,0,np.pi/2-eps,N=1000) for eps in epsilon])

In [ ]:
plt.plot(epsilon,intsqrttan)

In [50]:
# primer intento Cuadraturas de Gauss
def int_gauss3(f,x1,x2,x3,x4):
    mV = np.matrix([
        [1,1,1,1],
        [x1,x2,x3,x4],
        [x1**2,x2**2,x3**2,x4**2],
        [x1**3,x2**3,x3**3,x4**3]
    ])
    B = np.matrix([
        [2],
        [0],
        [2/3],
        [0]
    ])
    C=(np.linalg.inv(mV)*B).T
    print("Vector de C's:",C)
    area = C[0,0]*f(x1)+C[0,1]*f(x2)+C[0,2]*f(x3)+C[0,3]*f(x4)
    return area

In [2]:
def cos2(x):
    return np.cos(x)**2
def sec(x):
    return 1/np.cos(x)

In [59]:
int_gauss3(cos2,-0.339981043584856,-0.86113631159405,0.339981043584856,0.86113631159405)

Vector de C's: [[0.65214515 0.34785485 0.65214515 0.34785485]]


1.4546153354356177

In [60]:
int_gauss3(sec,-0.339981043584856,-0.86113631159405,0.339981043584856,0.86113631159405)

Vector de C's: [[0.65214515 0.34785485 0.65214515 0.34785485]]


2.451213216786557

In [58]:
int_gauss3(sec,-1,-np.pi/4,-np.pi/6,-np.pi/8)

Vector de C's: [[ -23.47373784   76.54333511 -126.72507116   75.65547389]]


0.3624466267688007

# Scipy para [Cuadraturas Gausianas](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.integrate.quadrature.html)

In [10]:
from scipy.integrate import quadrature
print(1+np.sin(2)/2)
quadrature(cos2,-1,1,maxiter=10)

1.454648713412841


(1.4546487134292152, 2.955303335383519e-09)

# Monte Carlo Integration

$$\pi=4\int_{0}^{1}\sqrt{1-x^2}\text{d}x$$

In [11]:
def f(x):
    return 4*np.sqrt(1-x**2)

In [16]:
x = np.random.uniform(size=1000000)
print("Monte Carlo pi:",f(x).mean())
del x

Monte Carlo pi: 3.140421150211313
